# CrewAI Multi-Agent Research & Writing System

This notebook demonstrates autonomous multi-agent orchestration for research and content creation using CrewAI framework.

## Overview
- **Framework**: CrewAI 0.80.0
- **LLM**: IBM WatsonX AI (LLaMA 3.3 70B Instruct)
- **Tools**: SerperDev API for web search
- **Workflow**: Sequential agent collaboration

## Agent Architecture
1. **Research Agent**: Senior Research Analyst with web search capabilities
2. **Writer Agent**: Tech Content Strategist for content creation

## Requirements
- Python 3.12+
- SerperDev API key for web search
- IBM WatsonX AI access (or alternative LLM)

In [ ]:
# Install necessary Python packages
!pip install langchain==0.3.20 | tail -n 1 
!pip install crewai==0.80.0 | tail -n 1
!pip install langchain-community==0.3.19 | tail -n 1 
!pip install crewai-tools==0.38.0 | tail -n 1
!pip install databricks-sdk==0.57.0 | tail -n 1

In [ ]:
# Set up environment variable for SERPER_API_KEY
import os
from getpass import getpass

# Load API key from environment or prompt user
if 'SERPER_API_KEY' not in os.environ:
    os.environ['SERPER_API_KEY'] = getpass("Enter your SERPER_API_KEY: ")

## Tool Configuration

Initialize SerperDev search tool for web research capabilities.

In [ ]:
# Import necessary libraries for search tool
from crewai_tools import SerperDevTool

In [ ]:
# Initialize the SerperDevTool for web search
search_tool = SerperDevTool()
print(f"Search tool type: {type(search_tool)}")

In [ ]:
# Test the search tool with a sample query
search_query = "Latest Breakthroughs in machine learning"
search_results = search_tool.run(query=search_query)

# Display search results
print(f"Search Results for '{search_query}':\n{search_results}")

## LLM Configuration

Configure IBM WatsonX AI with LLaMA 3.3 70B Instruct model.

In [ ]:
# Import LLM class from CrewAI
from crewai import LLM

In [ ]:
# Initialize the LLM with WatsonX AI parameters
llm = LLM(
    model="watsonx/meta-llama/llama-3-3-70b-instruct",
    base_url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
    max_tokens=2000,
)

## Agent Definition

Define specialized agents with distinct roles, goals, and capabilities.

### Research Agent

Senior Research Analyst with web search tool for gathering and analyzing information.

In [ ]:
# Import Agent class from CrewAI
from crewai import Agent

In [ ]:
# Define the Research Agent with specialized capabilities
research_agent = Agent(
    role='Senior Research Analyst',
    goal='Uncover cutting-edge information and insights on any subject with comprehensive analysis',
    backstory="""You are an expert researcher with extensive experience in gathering, analyzing, and synthesizing information across multiple domains. 
    Your analytical skills allow you to quickly identify key trends, separate fact from opinion, and produce insightful reports on any topic. 
    You excel at finding reliable sources and extracting valuable information efficiently.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[SerperDevTool()]
)

In [ ]:
# Display research agent configuration
research_agent

### Writer Agent

Tech Content Strategist for transforming research into engaging content.

In [ ]:
# Define the Writer Agent for content creation
writer_agent = Agent(
    role='Tech Content Strategist',
    goal='Craft well-structured and engaging content based on research findings',
    backstory="""You are a skilled content strategist known for translating 
    complex topics into clear and compelling narratives. Your writing makes 
    information accessible and engaging for a wide audience.""",
    verbose=True,
    llm=llm,
    allow_delegation=True
)

In [ ]:
# Display writer agent configuration
writer_agent

## Task Definition

Define specific tasks for each agent with clear objectives and expected outputs.

In [ ]:
# Import Task class from CrewAI
from crewai import Task

In [ ]:
# Define the Research Task for data gathering and analysis
research_task = Task(
    description="Analyze the major {topic}, identifying key trends and technologies. Provide a detailed report on their potential impact.",
    agent=research_agent,
    expected_output="A detailed report on {topic}, including trends, emerging technologies, and their impact."
)

In [ ]:
# Define the Writer Task for content creation
writer_task = Task(
    description="Create an engaging blog post based on the research findings about {topic}. Tailor the content for a tech-savvy audience, ensuring clarity and interest.",
    agent=writer_agent,
    expected_output="A 4-paragraph blog post on {topic}, written clearly and engagingly for tech enthusiasts."
)

## Crew Assembly & Execution

Assemble agents and tasks into a crew with sequential workflow processing.

In [ ]:
# Import Crew and Process classes from CrewAI
from crewai import Crew, Process

In [ ]:
# Initialize the Crew with agents, tasks, and sequential processing
crew = Crew(
    agents=[research_agent, writer_agent],
    tasks=[research_task, writer_task],
    process=Process.sequential,
    verbose=True
)

In [ ]:
# Execute the crew workflow with error handling
try:
    result = crew.kickoff(inputs={"topic": "Latest Generative AI breakthroughs"})
except Exception as e:
    print(f"Error during crew execution: {e}")
    raise

In [ ]:
# Check result type
type(result)

In [ ]:
# Display complete result object
result

## Output Analysis

Examine individual task outputs and agent performance.

In [ ]:
# Extract and display final output from the last agent
final_output = result.raw
print("Final output:", final_output)

In [ ]:
# Retrieve outputs from each task
tasks_outputs = result.tasks_output

In [ ]:
# Display research task output
print("Task Description:", tasks_outputs[0].description)
print("Output of research task:", tasks_outputs[0])

In [ ]:
# Display writer task output
print("Writer task description:", tasks_outputs[1].description)
print("\nOutput of writer task:", tasks_outputs[1].raw)

In [ ]:
# Display agent assignment for each task
print("Agent for researcher task:", tasks_outputs[0].agent)
print("Agent for writer task:", tasks_outputs[1].agent)

## Performance Metrics

Analyze token usage and computational cost.

In [ ]:
# Extract token usage metrics
token_count = result.token_usage.total_tokens
prompt_tokens = result.token_usage.prompt_tokens
completion_tokens = result.token_usage.completion_tokens

# Display detailed metrics
print(f"Total tokens used: {token_count}")
print(f"Prompt tokens: {prompt_tokens} (used for instructions to the model)")
print(f"Completion tokens: {completion_tokens} (generated in response)")